# Word2Vec實作
- 字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
- 之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
- 2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
- 本範例以維基百科wiki部分資料作範例
- 資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
- 利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
- 本範例約需1小時長時間執行


In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2

### opencc是繁簡轉換工具

In [ ]:
!pip install opencc-python-reimplemented

### gensim是訓練word2vec的函式庫

In [ ]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [ ]:
wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10]

## 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
text_num = 0  # 記錄處理的文本數量，初始化為0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:          # 開啟一個文本檔案 'wiki_text.txt' 以寫入模式，使用 UTF-8 編碼
    for text in wiki_corpus.get_texts():                 # 迴圈遍歷Wikipedia語料庫中的文本
        f.write(' '.join(text)+'\n')                  # 將每個文本中的單詞用空格連接成字串，然後寫入文件中，並在每個文本後換行
        text_num += 1                          # 更新處理的文本數量
        if text_num % 10000 == 0:                  # 每處理 10000 篇文章，輸出一次處理進度
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))            # 輸出最終處理的文本數量

In [ ]:
import jieba
from opencc import OpenCC


# Initial
cc = OpenCC('s2t')                               # 初始化 OpenCC，設定轉換為繁體中文
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()       # 開啟 'wiki_text.txt' 文本檔案，讀取內容
train_data = cc.convert(train_data)                      # 使用 OpenCC 將簡體中文轉換為繁體中文
train_data = jieba.lcut(train_data)                        # 使用 jieba 對文本進行分詞
train_data = [word for word in train_data if word != '']         # 過濾空字串
train_data = ' '.join(train_data)                        # 將分詞結果以空格連接成字串
open('seg.txt', 'w', encoding='utf-8').write(train_data)             # 將處理好的文本寫入 'seg.txt' 檔案

In [ ]:
from gensim.models import word2vec

# 設定模型的參數
# Settings
seed = 666          # 隨機種子，確保可重複性
sg = 0             # 使用 CBOW 模型（sg=0）或 Skip-gram 模型（sg=1）
window_size = 10        # 上下文窗口大小
#vector_size = 100
min_count = 1         # 最小詞頻門檻
workers = 8           # 訓練模型的平行化工作數量
#epochs = 5
batch_words = 10000      # 每個訓練批次的詞彙數

train_data = word2vec.LineSentence('seg.txt')      # 從 'seg.txt' 文本檔案中讀取訓練數據

# 使用 Word2Vec 建立詞向量模型
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')           # 儲存訓練好的 Word2Vec 模型

In [ ]:
from gensim.models import word2vec

string = '腦'                      # 指定詞語
model = word2vec.Word2Vec.load('word2vec.model')     # 載入訓練好的 Word2Vec 模型
print(string)

for item in model.wv.most_similar(string):         # 列印與指定詞語最相似的詞語及相似度
    print(item)